### 06 - Packaging

#### Outline
* Guidelines
* Recommended Stack & Template Package
* Dependency Mgmt
* Linting & Formatting
* Testing
* Typechecking
* Continuous Integration (CI)
* Repo Configuration
* Publishing

----
#### Guidelines

These evolve over time as the field of software continues to change.

Key points
* Quality only goes up: use automated tools to prevent backsliding and decay without excessive time commitment
* Use semver to reduce unexpected breaking changes
* Always keep track of units-of-measure
* Design reviews are important but don't need to be bureaucratic - just look over the design of large projects with a friend before you start work
* No new development in closed-source or obsolete languages except for interfacing with necessary dependencies in those languages

Notice that this doesn't include any specifics about _how_ these goals should be achieved. **The point isn't to use a tool, it's to produce quality software.** It does not matter what tools you use as long as your code is reliable, maintainable, documented, accessible, and performant.

That said, we can certainly recommend some specifics that are known to work.

----
#### Recommended Stack & Template Package

Python package management tools are fairly fragmented.<br>As a notable contrast, in Rust, all of these functions and more are covered by a single tool (cargo).

| Tool | Purpose |
|------|---------|
|github or gitlab| version control & CI runner |
| uv | dependency mgmt |
| hatchling | build backend |
| ruff | linting & formatting |
| pytest<br>pytest-cov<br>mktestdocs | testing |
| pyright | typechecking |
| mkdocs<br>mkdocs-material<br>mkdocstrings[python] | documentation |
| twine | publishing |

In addition, there are some common tasks that approach the formality of package management.<br>
My recommendations:

| Library | Purpose |
|---------|---------|
| click   | Command Line Interface (CLI) |
| plotly dash | Graphical User Interface (GUI) |
| rich | Formatted logging |

----

#### Step-By-Step Example

##### Initialize
1. Make a project folder
    * `uv init --lib --no-pin-python <PROJECT_NAME>`
2. Add dev-dependencies to pyproject.toml
    * Include everything needed for QC/CI tools (linting, testing, docs, etc)
    * Usually just copy this block forward from another project
3. Add `tests`, `examples`, and `docs` folders
    * Add an empty `.placeholder` file in each so that they can be added to the repo
4. Add `__version__` parameter in package level `__init__.py`

##### Set up QC
5. Add linter configuration to `pyproject.toml`
6. Add coverage fail-under to `pyproject.toml`
7. Add github actions test workflow
    * Typecheck, lint, test, build docs
8. Add doctesting function in `tests/test_docs.py`
9. Add example testing function in `tests/test_examples.py`

##### Add Docs
9. Copy forward a `mkdocs.yml` and `readthedocs.yml` from another repo
10. Make sure the API doc autogenerator is enabled (mkdocstrings[python])
11. Update index.md, api.md, and page routing
    * Note - items without docstrings will not appear in the docs at all!

##### Add Logging
12. Add a simple logging layer
    * At minimum, include timestamps and stack origin of messages & allow writing to a logfile
    * Log major branches in the code at `info` level
    * Log suspicous input, output, or internal state at `warning` level
    * Log recovery from clearly incorrect state at `error` level

##### Add CLI
13. Add an executable entrypoint in `pyproject.toml` like
```toml
[project.scripts]
refprpoj = "refproj.cli:cli"
```
14. Add a `cli.py` file in the source folder
15. Add a `cli()` function in cli.py and populate it with click decorators
16. Add a test of the CLI using click CliRunner

##### Add GUI
17. Add a `gui.py` source file
18. Add a simple dash app in gui.py
19. Add a `refproj gui` CLI command

##### Set up Github Repo
20. Make a new github repo
21. Push the existing project to the repo
22. Enable actions workflows in the repo settings
23. Enable branch protections for `main` branch in repo settings
    * Require PRs
        * Require passing CI workflows to merge
        * Require up-to-date branch before merge
    * Do not allow deletion
    * Do not allow bypassing rules (this can lead to accidental deletion or modification by admins)

##### Finishing Touches
24. Add a `CHANGELOG.md` (see other changelogs for examples of formatting conventions)
    * Update this changelog with every version
    * To reuse the effort of making the changelog entry, use it as the PR description and PR merge commit message


----
#### Version Control, CI, and Repo Configuration

* Both github and gitlab combine version control, access management, and job-running systems
* Honorable mention: Codeberg provides version control and access management, but does not have job-runners
  * Instead, it can be integrated with 3rd-party job runners like CircleCI, Jenkins, etc
* **Use your CI system to run typechecking, linting, and unit tests**

##### Recommended Configuration & Workflows

* Versioning strategy
  * Semantic versioning is the only responsible choice
  * Others are reliability issues
  * Maintain a CHANGELOG.md (and reuse the entries for commit messages and PR descriptions)
* Commit workflow
  * Every commit on `main` is a valid, fully-functioning version of the software
  * Commit however often and in whatever way you like on branches
  * Squash branch to a single commit before merging to main
  * Commit messages on `main` should be the changelog entry
* Branch protections for `main`
  * No deletion
  * Require PRs (even if you are working solo!)
  * Require 1 PR approver (ok to enable this after the initial period of rapid development)
  * Do not allow admins to bypass rules
    * If it's an emergency, they can temporarily switch the configuration
    * Disallowing bypass prevents accidental pushes to main
* PR workflows
  * Working solo: Squash-Merge Only
    * Squash-merge breaks connection between `main` and branches; only works if there is only one person working on the project at a time
  * With a team: Manually squash branch before merging
    * This preserves continuity of history on `main`
  * Long-term dev projects: Add a `develop` branch with relaxed quality standard until the smoke clears
  * Either way,
    * Configure repo to require passing test workflows to merge PR
    * Configure repo to require up-to-date branch before merging
      * When you merge a PR, the new state of main should be exactly the same as the state of the PR branch! Otherwise, how do you know it isn't going to be broken after the auto-merge?
    * PR description can be the changelog entry


----

#### Dependency Mgmt

* uv is faster and more reliable than the other options
  * They maintain their own pypi metadata database to accelerate solving dependency graph
  * Pip by itself can stall semi-permanently on a difficult configuration due to needing to download entire packages to check dep compatibility
  * uv can also handle your python installation!
* Honorable mention
  * Poetry is fairly solid, but slower, less reliable, and less featured than uv
  * Just using a bare pyproject.toml is a valid option
    * Dep managers provide extra features like dep locking, but aren't strictly required

A new project can be initialized with uv by doing

```bash
uv init --lib --no-pin-python --build-backend=setuptools <MY_PACKAGE_NAME>
```

with `<MY_PACKAGE_NAME>` replaced with your choice of package name.

Then, use uv with your package like:

```bash
uv sync  # Update lockfile
uv pip install -e .  # Install editable dev version
uv build  # Generate distribution artifacts (wheel, sdist)
uv run --locked <COMMAND>  # Run a command in a uv env that has locked deps (makes CI reliable)
```


----

#### Build Backend

Build backends handle assembling your library into a packaged `wheel` (binary) or `sdist` (source) distribution artifact.

`uv` defaults to `hatchling`, but as of 2025-06-30 this is still causing sporadic import resolution problems with unknown root cause. To resolve this, setuptools (the original build backend) should be used instead for now.

If Rust language bindings are used, the `maturin` build backend can be used to manage compiling and linking PyO3 Rust bindings. The maturin project also provides a github action template for cross-compiling and publishing for different platforms, making it essentially effortless to publish mixed Rust-Python projects.

----
#### Linting & Formatting

"Linting" is essentially spell-checking for code; formatting automatically resolves most linting errors. If you don't lint and format your code, it will look bad, it will be difficult to read, and it will not inspire confidence or trust.

Traditionally, Python linters and formatters have been separate entities, and careful configuration was required to produce a self-consistent combination of linter and formatter. This was folly.

Ruff provides both linting and formatting, and formats code in a way that is consistent with its own linting. It is also substantially faster than any previous linter or formatter. There is no reason to use another tool for this.

Use like

```bash
ruff format .
ruff check . --fix  # Auto-fix structural lints
ruff check .  # Check for remaining structural lints
```

By default, `ruff check` only errors on structural issues, not formatting.<br>
Formatting lints can be selected by [adding configuration to the `pyproject.toml`](https://docs.astral.sh/ruff/linter/#__tabbed_2_1):



----
#### Typechecking

Python does not natively check correctness of types at runtime,
and the language itself provides no tools for checking statically.
Instead, an external tool is needed.

It is recommended to use a typechecker as early in a project as possible.
Without appropriate guidance, it's easy to end up with thousands of type errors
that make it difficult to add typechecking late in development.

Typecheckers
* `pyright`: recommended for now
  * Fairly good type inference; not unnecessarily facetious
  * Ironically, `pyright`'s package versioning is awful - it regularly has unexpected breaking changes caused by internal javascript deps that can't be pinned by a python dependency manager
* `mypy`: works, but incredibly punishing
  * Does not infer types; requires complete type hints
  * Makes some sections of the language unusable
* `ty`: very promising in the future
  * Switch to `ty` as soon as it's stable
  * It is under construction now, but already maintains a higher standard of quality and performance than other tools
  * Just missing some features as of 2025-06-30

Run pyright like

```bash
pyright .
```

----
#### Testing

In addition to obvious reliability benefits, testing provides progress capture: once you get something working, if it's tested, then it will continue to work in the future. Otherwise, features tend to go stale as things change, they become broken, and go unfixed for long periods of time until the developers forget what caused the breakage and what is needed to fix it.

`pytest` provides basic functionality to find and run unit tests.

`pytest-cov` extends this to check which lines run and which do not, in order to show gaps in testing.

`mktestdocs` provides doctesting, which finds and runs code segments in documentation to ensure that documentation does not go stale.

---

Coverage testing can run automatically alongside pytest by setting configuration in `pyproject.toml`:

```toml
[tool.pytest.ini_options]
addopts = "--cov=<project-name> --cov-report html --cov-fail-under=90"
```

The fail-under percentage limit is adjustable. 90% is typically a good number to aim for non-safety-critical code. 100% coverage is commendable, but not necessarily the most efficient balance of developer time for every package.

Coverage fail-under should
not be decreased casually to avoid writing tests. That said, it may be a valid strategy if the newly-added context is extraordinarily difficult to test (for example, GUIs and CLIs frequently do not yield easily to testing). Decreasing coverage percentage is better than writing testable fluff to pad the numbers.

---

##### Always test your docs!

Examples embedded in documentation should be up-to-date at all times, and this can be ensured by using a tool (`mktestdocs`) to find and run code segments in documentation.

---

##### Always test your examples!

There is no excuse for standalone examples to go stale and stop working. Examples should be included in testing so that if a change to the code breaks and example, that change can't be merged into main.

Below is a method for testing standalone examples.

```python

import os
import pathlib
import runpy

import pytest

EXAMPLES_DIR = pathlib.Path(__file__).parent / "../examples"
EXAMPLES = [
    EXAMPLES_DIR / x
    for x in os.listdir(EXAMPLES_DIR)
    if (os.path.isfile(EXAMPLES_DIR / x) and x[-3:] == ".py")
]


@pytest.mark.parametrize("example_file", EXAMPLES)
def test_example(example_file: pathlib.Path):
    print(example_file)
    runpy.run_path(str(example_file), run_name="__main__")

```

----
#### Continuous Integration (CI)

CI historically has a more specific meaning, but now usually refers to the practice of using job-runner clusters to run linting, testing, and publishing of packages automatically.

CI configuration varies by provider, but there are common themes. Ultimately, you will rarely write those configs by hand, except sections that represent a bash script.

Setting them up involves quite a bit of copy-pasting and troubleshooting, and unfortunately, because the config formats change rapidly and are only just starting to see the appearance of proper language support, AI coding assistants are not yet effective at relieving the boilerplate grind of configuring CI systems.

Some pointers for setting up github or gitlab CI
* Configure the repo to require passing tests for merging to main
* Start from a template, use an auto-generation tool, or copy forward a working config from another repo
* Look for examples or prepackaged workflows before handwriting custom ones


----
#### Documentation

`mkdocs` provides a simple, easy-to-configure, easy-to-write interface for documentation using the ever-popular markdown-with-html format.

`mkdocstrings[python]` provides **automatic generation of formatted API docs from your code's docstrings**. This is a very powerful feature, and an excellent way to reuse the effort put into writing docstrings. This also provides a guarantee that the docs will remain up to date with the code at all times.

`mkdocs-material` provides excellent styling for mkdocs.

`mktestdocs` provides doctesting for markdown files by crawling the files for codeblocks.

**Readthedocs** can be used to host mkdocs documentation by pointing it toward the repository. It is best used for public repos, and has only minimal functionality for private doc pages. Github Pages can also be used to host mkdocs sites, and has better features for supporting private company-internal pages.

mkdocs has a convenient single-file-config format. That said, it can be hard to start it from scratch, so it's usually best to copy forward a working config from an existing project as long as you know what every part of the config does.

----
#### Publishing

`twine` is a simple, popular, and reliable tool for publishing distribution artifacts (wheels & sdist) to a package repo.

##### Never keep credentials for the public pypi on your local machine

Add them as environment secrets in your CI provider and publish from there. That way, there is no possibility of accidentally publishing anything from your local machine, which is especially important if you work with both public and private projects.